# 14. Defining Classes

## Generics
---

If a method needs additional information to operate properly, this information is passed to the method using parameters.    
   
During the execution of the program, when calling this particular method, we pass arguments to the method, which are assigned to its parameters and then used in the method’s body.   

Like with methods, when we know that the functionality (actions) encapsulated into a `class` can be applied not only to objects of one, but to **many** (**heterogeneous**) **types**, and even in the scenario that these **types should not be known at the time of declaring the class**, we can use a functionality of $C\#$ called **generics**.
   
**Generics** allows us to declare parameters of a given class by **indicating an unknown type** that the class will eventually use. Then, when we instantiate our **generic** class, we **replace the unknown type with a specific one**. 
   
Accordingly, the newly created object will **only work with objects of this type** that we have assigned at its initialization. The specific type can be **any data type** that the compiler recognizes, including `class`, `struct`, `enum`, or another **generic** class.

<br>

### Why Use Generics?

To illustrate an applicable use case for **generics**, suppose we have declared the following class representing a **Dog**:

In [1]:
public class Dog
{
    public override string ToString()
    {
        return "WOOF!";
    }
}

<br>

So let's say we set out to define a class that represents a **shelter for homeless animals**, containing a specific number of free kennels which determine the capacity of animals that can find refuge in the shelter. Let's also say that one important caveat to  class that we want to create is that it needs to **only accommodate animals of the same kind**, in our case, `Dog`s only.

In [2]:
public class AnimalShelter
{

    // FIELD DEFINITIONS ---------------------------------------------------------------------
    private const int DEFAULT_CAPACITY = 20;
    private int placesOccupied;
    public Dog[] kennels;
    //----------------------------------------------------------------------------------------


    // CONSTRUCTORS --------------------------------------------------------------------------
    // The first constructor is parameterless, 
    // but uses constructor chaining to pass the DEFAULT_CAPACITY as an argument
    public AnimalShelter() : this( DEFAULT_CAPACITY )
    {
    }

    // Another constructor initializes the Dog Array 
    // to reflect a capacity which is defined by the numberOfPlaces argument
    public AnimalShelter( int numberOfPlaces )
    {
        this.kennels = new Dog[ numberOfPlaces ];
        this.placesOccupied = 0;
    }
    //----------------------------------------------------------------------------------------


    // METHODS -------------------------------------------------------------------------------
    // Add a New Dog to the Dog Array, provided that there is still enough space  
    public void Shelter( Dog newDog )
    {

        if ( this.placesOccupied <= this.kennels.Length)
            this.kennels[ this.placesOccupied++ ] = newDog;
            
    }

    // Remove a Dog from the Dog Array and returns that removed Dog, 
    // provided that the given Dog Array Index is valid 
    public Dog Release( int dogIndex )
    {

        if ( dogIndex < 0 || dogIndex >= this.placesOccupied )
        {
            throw new ArgumentOutOfRangeException(
                $"Index {dogIndex} is an invalid selection."
            );  
        }


        Dog releasedDog = this.kennels[ dogIndex ];


        for( int currentIndex = dogIndex; currentIndex < this.placesOccupied - 1; currentIndex++ )
        {
            this.kennels[ currentIndex ] = this.kennels[ currentIndex + 1 ];
        }


        this.kennels[ (this.placesOccupied--) - 1 ] = null;


        return releasedDog;

    }
    //----------------------------------------------------------------------------------------

}

<br>

Below, let's instantiate a Shelter, specifically for `Dog`s, with enough kennel space to fit up to 5:  

In [3]:
AnimalShelter dogShelter = new AnimalShelter(5);

<br>

Now, let's add enough new `Dog`s to our `dogShelter` to fill every kennel except the last:

In [4]:
for( int kennelIndex = 0; kennelIndex < dogShelter.kennels.Length - 1; kennelIndex++ )
    dogShelter.Shelter( new Dog() );

<img src="_img/InitDogKennel.png" style="width: 600px; display: block; margin: auto"></img>

In [5]:
foreach( Dog kennel in dogShelter.kennels )
{
    Console.WriteLine(  (kennel != null) ? kennel : "<null>" );
}

WOOF!
WOOF!
WOOF!
WOOF!
<null>


<br>

Now suppose we wanted to Release the `Dog` in the 2nd kennel:

<img src="_img/DogKennelRelease1.png" style="width: 600px; display: block; margin: auto"></img>

In [6]:
Dog releasedAnimal = dogShelter.Release( 1 );

<img src="_img/DogKennelRelease2.jpg" style="width: 600px; display: block; margin: auto"></img>

$$\Huge{\Downarrow \Downarrow}$$

$$\Huge{\Downarrow \Downarrow}$$

<img src="_img/DogKennelRelease3.jpg" style="width: 600px; display: block; margin: auto"></img>

In [7]:
foreach( Dog kennel in dogShelter.kennels )
{
    Console.WriteLine(  (kennel != null) ? kennel : "<null>" );
}

WOOF!
WOOF!
WOOF!
<null>
<null>


In [8]:
releasedAnimal

WOOF!

<br>

So, while the process above has worked fairly well so far, what would happen if we instead wanted to use our `AnimalShelter` to store instances of the following `Cat` `class`:

In [9]:
public class Cat
{
    public override string ToString()
    {
        return "MEOW!";
    }
}

<br>

Consequently, if we want to create an `AnimalShelter` for `Cat`s, we will **not be able to reuse the class that we already created**, although the operations of adding and removing animals from the shelter will be identical.    
   
Therefore, we have to literally copy the `AnimalShelter` class, and change only the type of the objects which are handled.   

Otherwise, we would run into the following error:

In [10]:
AnimalShelter catShelter = new AnimalShelter(5);

catShelter.Shelter( new Cat() )

Error: (3,21): error CS1503: Argument 1: cannot convert from 'Cat' to 'Dog'

<br>

As such, we can see that this solution of the problem is **not sufficiently comprehensive** as it does not provide a single class that describes our shelter for **any kind of animal** (i.e. for all objects) of the same type.

<br>

<br>

### Declaration of Generic Classes

Formally, the parameterizing of a class is done by adding `<T>` to the declaration of the class, after its name, where `T` is the **substitute** (**parameter**) of the **type** which will be used later:

```c#
[access_modifers] class ClassName <T>
{
    // class implementation goes here...
}
```

It should be noted that the characters `<` and `>`, which surround the **substitution** `T`, are an obligatory.

<br>

A particular class may have **more than one substitute** (to be parameterized by **more than one type**), depending on its needs.   
    
If the class needs several different unknown types, these types should be listed by a comma between the characters `<` and `>` in the declaration of the class, as each of the **substitutes** used must be **different identifier** (e.g. a **different letter**) – in the definition they are indicated as `T`, `U`, … , `N`.

```c#
[access_modifers] class ClassName <T [, U, ... , N]>
{
    // class implementation goes here...
}
```

<br>

<br>

### Specifying Generic Classes

The instantiation of **generic** classes should be done as generilzed below:

```c#
ClassName <Type> varliableName = new ClassName <Type> ();
```

Following the above generalization, if we want to create two shelters, one for `Dog` types and one for `Cat` typess, we should do so as follows:

```c#
AnimalShelter <Dog> dogShelter = new AnimalShelter <Dog> ();
AnimalShelter <Cat> catShelter = new AnimalShelter <Cat> ();
```

<br>

#### Using Unknown Types When Declaring Fields

Once used during the class declaration, the **substitute parameters** are used to indicate the **unknown types** that are visible in the whole body of the class, therefore they can be used to **declare the class's fields** as having the **same substitute type**, as demonstrated below:

```c#
[access_modifers] T fieldName;
```

<br>

#### Using Unknown Types When Declaring Methods

Similarly we may use **substitute parameters** in the declaration of **methods**, namely:

- As a **parameter** in the **list of parameters** of the method:
  - `returnType MethodName( T parameterName )`   

- As a **result** of the method's implementation:
  - `T MethodName( [parameters] )`

When the **type of method’s parameters cannot be specified**, we can **parameterize** (**typify**) the method.   
    
Accordingly, the indication of a specific type will happen during the **invocation** of the method, replacing the unknown type with a specific one, as we did in the classes.    

**Typifying** of a method is done, when *after the name* and *before the opening bracket* of the method, we add `<K>`, where `K` is the **replacement of the type** that will be used later:

```c#
returnType MethodName <K> ( [parameters] )
```

Accordingly, we can use **unknown type**, `K`, for: 
- parameters in the parameter’s list of method
- the **return value**, 
- **declare variables of type substitute** `K` in the body of the method.

<br>

##### Typifying of a Generic Method - Example

For example, consider a method that **swaps the values of two variables**:

In [11]:
public void Swap<K>(ref K a, ref K b)
{
    K oldA = a;
    a = b;
    b = oldA;
}

<br>

This is a method that **works without regard of the types** given to it as arguments.   
For that reason,we define it as a **generic**.

In [12]:
public int five = 5,
           ten  = 10;


Swap<int>( ref five, ref ten );


Console.WriteLine( $"After the Swap, five = {five} and ten = {ten}")

After the Swap, five = 10 and ten = 5


In [13]:
public string firstWord  = "First Word",
              secondWord = "Second Word"; 


Swap<string>( ref firstWord, ref secondWord );


Console.WriteLine( 
    $"After the Swap, 1st_Word = {firstWord} and 2nd_Word = {secondWord}"
)

After the Swap, 1st_Word = Second Word and 2nd_Word = First Word


<br>

we can also **omit the explicit declaration of a specific type**, because the compiler will detect it automatically, recognizing the type of the given parameters.

In [14]:
Swap( ref five, ref ten );

Console.WriteLine( $"After the Swap, five = {five} and ten = {ten}")

After the Swap, five = 5 and ten = 10


In [15]:
Swap( ref firstWord, ref secondWord );

Console.WriteLine( 
    $"After the Swap, 1st_Word = {firstWord} and 2nd_Word = {secondWord}"
)

After the Swap, 1st_Word = First Word and 2nd_Word = Second Word


<br>

<br>

### Putting It All Together - A Generic `AnimalShelter` Class 

We are now ready to redesign our `AnimalShelter` class to support `Cat` types, in addition to `Dog` types:

In [16]:
public class AnimalShelter <T>
{

    // FIELD DEFINITIONS ---------------------------------------------------------------------
    private const int DEFAULT_CAPACITY = 20;
    private int placesOccupied;
    public T[] kennels;
    //----------------------------------------------------------------------------------------


    // CONSTRUCTORS --------------------------------------------------------------------------
    // The first constructor is parameterless, 
    // but uses constructor chaining to pass the DEFAULT_CAPACITY as an argument
    public AnimalShelter() : this( DEFAULT_CAPACITY )
    {
    }

    // Another constructor initializes the Animal Array 
    // to reflect a capacity which is defined by the numberOfPlaces argument
    public AnimalShelter( int numberOfPlaces )
    {
        this.kennels = new T[ numberOfPlaces ];
        this.placesOccupied = 0;
    }
    //----------------------------------------------------------------------------------------


    // METHODS -------------------------------------------------------------------------------
    // Add a New Animal to the Animal Array, provided that there is still enough space  
    public void Shelter( T newAnimal )
    {

        if ( this.placesOccupied <= this.kennels.Length)
            this.kennels[ this.placesOccupied++ ] = newAnimal;
            
    }

    // Remove a Dog from the Animal Array and returns that removed Dog, 
    // provided that the given Animal Array Index is valid 
    public T Release( int animalIndex )
    {

        if ( animalIndex < 0 || animalIndex >= this.placesOccupied )
        {
            throw new ArgumentOutOfRangeException(
                $"Index {animalIndex} is an invalid selection."
            );  
        }


        T releasedAnimal = this.kennels[ animalIndex ];


        for( int currentIndex = animalIndex; currentIndex < this.placesOccupied - 1; currentIndex++ )
        {
            this.kennels[ currentIndex ] = this.kennels[ currentIndex + 1 ];
        }


       // The line below will cause an error 
       // since we are trying to use the default value for a reference type, 
       // but we are not sure whether this type is a reference type or a primitive:
       
            // this.kennels[ (this.placesOccupied--) - 1 ] = null;

        // As such, to avoid this issue
        // we should replace 'null' with the default(T) construct:
        this.kennels[ (this.placesOccupied--) - 1 ] = default(T);


        return releasedAnimal;

    }
    //----------------------------------------------------------------------------------------

}

<br>

Below, let's instantiate a Shelter, specifically for `Cat`s, with enough kennel space to fit up to 5:  

In [17]:
AnimalShelter <Cat> catShelter = new AnimalShelter <Cat> (5);

<br>

Now, let's add enough new `Cats`s to our `catShelter` to fill every kennel except the last:

In [18]:
for( int kennelIndex = 0; kennelIndex < catShelter.kennels.Length - 1; kennelIndex++ )
    catShelter.Shelter( new Cat() );

In [19]:
foreach( Cat kennel in catShelter.kennels )
{
    Console.WriteLine(  (kennel != null) ? kennel : "<null>" );
}

MEOW!
MEOW!
MEOW!
MEOW!
<null>


<br>

Now suppose we wanted to Release the `Cat` in the 2nd kennel:

In [20]:
Cat releasedAnimal = catShelter.Release( 1 );

In [21]:
foreach( Cat kennel in catShelter.kennels )
{
    Console.WriteLine(  (kennel != null) ? kennel : "<null>" );
}

MEOW!
MEOW!
MEOW!
<null>
<null>


<br>

As demonstrated above, we have now adapted the `AnimalShelter` class to accept animal objects of **any type**.